In [ ]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import StringPromptTemplate
from typing import List, Dict, Optional
import asyncio
import paramiko
import json
from pydantic import BaseModel, Field

class SSHConnection:
    def __init__(self, hostname: str, username: str, password: str = None, key_filename: str = None):
        self.client = paramiko.SSHClient()
        self.client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        self.client.connect(
            hostname=hostname,
            username=username,
            password=password,
            key_filename=key_filename
        )

    async def execute_interactive(self, command: str, timeout: int = 30) -> asyncio.Queue:
        """
        Execute an interactive command and return a queue for streaming output
        """
        output_queue = asyncio.Queue()
        
        # Start interactive shell session
        shell = self.client.invoke_shell()
        
        async def read_output():
            while True:
                if shell.recv_ready():
                    output = shell.recv(4096).decode('utf-8')
                    await output_queue.put(output)
                await asyncio.sleep(0.1)
                
        # Start background task to read output
        asyncio.create_task(read_output())
        
        # Send command
        shell.send(command + '\n')
        
        return output_queue

class LinuxCommandTool(Tool):
    def __init__(self, ssh_connection: SSHConnection):
        self.ssh = ssh_connection
        super().__init__(
            name="linux_command",
            description="Execute Linux commands and handle interactive output",
            func=self.run
        )
        
    async def run(self, command: str) -> str:
        output_queue = await self.ssh.execute_interactive(command)
        
        # Collect output for a reasonable time
        full_output = []
        try:
            while True:
                output = await asyncio.wait_for(output_queue.get(), timeout=1.0)
                full_output.append(output)
                
                # Check if command has completed (can be customized based on needs)
                if output.strip().endswith('$ '):  # Basic prompt detection
                    break
        except asyncio.TimeoutError:
            pass
            
        return ''.join(full_output)

class CommandState(BaseModel):
    """Track the state of command execution"""
    command: str = Field(description="The command being executed")
    output: str = Field(description="Current accumulated output")
    status: str = Field(description="Current status: running/completed/error")
    next_action: Optional[str] = Field(description="Next action to take based on output")

class InteractiveAgentPrompt(StringPromptTemplate):
    template = """You are an expert Linux system administrator.
    Current command state: {current_state}
    
    Based on the command output, determine the next action:
    1. If the command is complete, analyze the output and summarize the results
    2. If the command requires interaction, provide the next input
    3. If there's an error, suggest how to resolve it
    
    Previous actions: {memory}
    
    Your response should be in JSON format:
    {{"action": "complete/interact/error",
      "response": "your analysis or next input",
      "reasoning": "your reasoning"}}
    """
    
    def format(self, **kwargs) -> str:
        return self.template.format(**kwargs)

class InteractiveAgent:
    def __init__(self, llm, tools: List[Tool]):
        self.llm = llm
        self.tools = tools
        self.memory = ConversationBufferMemory()
        self.prompt = InteractiveAgentPrompt()
        
    async def run(self, command: str):
        state = CommandState(
            command=command,
            output="",
            status="running",
            next_action=None
        )
        
        while state.status == "running":
            # Get next action from LLM
            llm_response = await self.llm.apredict(
                self.prompt.format(
                    current_state=state.dict(),
                    memory=self.memory.buffer
                )
            )
            
            action_data = json.loads(llm_response)
            
            if action_data["action"] == "interact":
                # Send next input to command
                for tool in self.tools:
                    if tool.name == "linux_command":
                        new_output = await tool.run(action_data["response"])
                        state.output += new_output
                        
            elif action_data["action"] == "complete":
                state.status = "completed"
                return action_data["response"]
                
            elif action_data["action"] == "error":
                state.status = "error"
                return action_data["response"]
            
            # Update memory
            self.memory.save_context(
                {"input": state.command},
                {"output": action_data["reasoning"]}
            )

# Example usage
async def main():
    # Initialize SSH connection
    ssh = SSHConnection(
        hostname="example.com",
        username="user",
        key_filename="~/.ssh/id_rsa"
    )
    
    # Create tools
    tools = [LinuxCommandTool(ssh)]
    
    # Initialize LLM
    llm = ChatOpenAI(temperature=0)
    
    # Create agent
    agent = InteractiveAgent(llm, tools)
    
    # Run interactive command
    result = await agent.run("sudo apt update")
    print(f"Final result: {result}")

if __name__ == "__main__":
    asyncio.run(main())